In [8]:
import os
from tqdm import tqdm
from datasets import load_dataset
import sys
sys.path.append('../evaluation')
from evaluate import RetrievalSystem, main as evaluate_main
import yaml
import hyde

In [4]:
with open("/users/christineye/retrieval/config.yaml", 'r') as stream:
    api_key = yaml.safe_load(stream)['openai_api_key']

In [2]:
%load_ext autoreload
%autoreload 2

In [45]:
test_hyde = hyde.HydeRetrievalSystem(config_path = "/users/christineye/retrieval/config.yaml", 
                                     db_path = "")